In [1]:
import numpy as np

# The units package
import pint 
# you may need to "pip install pint"

import scipy.constants
from scipy.constants import physical_constants

In [2]:
# This gives you the default base units, SI
units = pint.UnitRegistry() 

s    = units.second
m    = units.metre
kg   = units.kilogram

# We can use fancy unicode symbols too
Å    = units.angstrom
Ω    = units.ohm
ftnt = units.fortnight

In [3]:
# Basics
speed = 10*m/s

print(speed)
print(speed.units)
print(speed.dimensionality)

print(speed.to(Å/ftnt))

10.0 meter / second
meter / second
[length] / [time]
1.2096e+17 angstrom / fortnight


In [4]:
# This makes it easy to use constants from scipy
def make_constant(name,use_uncertainty=False):
    '''Make a dimensional quanitity from a CODATA value using pint and scipy'''
    if not (name in physical_constants):
        matches = scipy.constants.find(name)
        if matches==[] :
            raise LookupError("Constant not found, use scipy.constant.find(name) \n or see https://docs.scipy.org/doc/scipy/reference/constants.html")
        else:
            from warnings import warn
            warn("Assuming \"{}\" means \"{}\".".format(name,matches[0]))
            name = matches[0]
    value, unit, error = physical_constants[name] 
    if use_uncertainty:
        return value*units.parse_expression(unit).plus_minus(error)
    else:
        return value*units.parse_expression(unit)

In [5]:
# E&M Example

μ0 = make_constant("mag. constant")
ε0 = make_constant("electric constant")
c  = make_constant("speed of light") # This isn't the exact name so it will warn

# Can use numpy functions with dimensional quantities
c1 = 1/np.sqrt(μ0*ε0)

# The print formatting is controlled like {:format}
# where ~ means abbreviate, L means LaTeX, P means pretty

print("μ₀ = {:~P}".format(μ0))

print()

print(
    "Griffiths says that c = {:~}".format(c),
    "should equal \n 1/√(μ₀ε₀) = {:~}".format(c1.to(m/s)),
)

print()

error = 100*(np.abs(c1-c)/c)
if error.dimensionless:
    print("Relative error: Δc/c = {}".format(error.magnitude))

μ₀ = 1.2566370614359173e-06 N/A²

Griffiths says that c = 299792458.0 m / s should equal 
 1/√(μ₀ε₀) = 299792457.99999994 m / s

Relative error: Δc/c = 1.9881969404110434e-14


C:\Users\danny\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Assuming "speed of light" means "speed of light in vacuum".


In [6]:
# A fancy example with LaTex and uncertainties
# you may need to "pip install uncertainties"

from IPython.core.display import Latex #, HTML, Math, display

G = make_constant("newton",use_uncertainty=True)

# Latex strings in python must use double braces like \mathrm{{abc}}
# so the dont contflict with things like "x={}".format(y)
Latex(r'''
    The gravitational constant $G$ has dimensions ${:L}$ and hence can be expressed in
    terms of Planck length, mass and time as, 
    $$G= \frac{{l_{{\rm P}}^3}}{{m_{{\rm P}} t_{{\rm P}}^2}} =  {:~L}$$
    '''.format(G.dimensionality, G))

C:\Users\danny\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Assuming "newton" means "Newtonian constant of gravitation".


<IPython.core.display.Latex object>